In [36]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss
import tensorly as tl
tl.set_backend('jax')
# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

In [25]:
sample = np.genfromtxt('references/sample.csv', delimiter=',')
normalized_sample = jnp.transpose(sample/sample[0])
print(type(normalized_sample))
print(normalized_sample.shape)

<class 'jaxlib.xla_extension.DeviceArray'>
(1000, 25)


In [31]:
# S = jnp.array([
#             [1.00, 1.09, 1.08, 1.34],
#             [1.00, 1.16, 1.26, 1.54],
#             [1.00, 1.22, 1.07, 1.03],
#             [1.00, 0.93, 0.97, 0.92],
#             [1.00, 1.11, 1.56, 1.52],
#             [1.00, 0.76, 0.77, 0.90],
#             [1.00, 0.92, 0.84, 1.01],
#             [1.00, 0.88, 1.22, 1.34]])
S = normalized_sample

In [32]:
N = S.shape[1]          # number of time steps
r = 0.06       # interest rate
K = 1.1        # strike 
T = 1          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [33]:
H = np.maximum(K - S, 0)           # intrinsic values for put option

V = np.zeros_like(H)               # value matrix
V[:,-1] = H[:,-1]

# Valuation by LS Method
for t in range(N-2, 0, -1):

    good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
                                   # the regression is performed only on these paths 
    
    rg = jnp.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
    C = jnp.polyval( rg, S[good_paths,t] )                             # evaluation of regression  
    
    exercise = np.zeros( len(good_paths), dtype=bool)    # initialize
    exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
    V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
    V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
    discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
    V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
print("Example price= ", V0)

Example price=  0.4206863


In [34]:
print(H.round(2))


[[0.1  0.   0.   ... 0.42 0.28 0.38]
 [0.1  0.29 0.45 ... 0.48 0.5  0.38]
 [0.1  0.35 0.41 ... 0.24 0.25 0.28]
 ...
 [0.1  0.26 0.07 ... 0.   0.   0.  ]
 [0.1  0.04 0.   ... 0.   0.09 0.15]
 [0.1  0.05 0.01 ... 0.95 0.91 1.01]]
